Write `.bashrc`

In [1]:
%%writefile ~/.bashrc
# Source global definitions
if [ -f /etc/bashrc ]; then
        . /etc/bashrc
fi

# User specific aliases and functions
# Juan's heritage
PS1="\u\[\033[m\]@\[\033[32m\]\h:\[\033[33;1m\]\W\[\033[m\]$ " # para colorear la consola
#alias ls='ls --color ' #Para colorear los directorios
alias ll='ls --color -lah '
alias lt='ls --color -lta '
alias rm='rm -i'
alias grep='grep --color=auto'
alias tarzip='tar -cvzf'
alias untar='tar -xvzf'
alias rmi='rm -i'



Writing /home/jovyan/.bashrc


In [4]:
!pip install seqspec

In [103]:
%%bash
mkdir -p data

In [ ]:
import numpy as np
import pandas as pd
df = pd.read_csv("/mnt/shared/counting_guides/metadata/TF_CRISPR_i_a.scRNAseq_screen.igvf_metadata.txt", sep = '\t')

def get_index_str(c):
    if c.targeting:
        return f"{c['guide_chr']}_{int(c['guide_start'])}_{int(c['guide_end'])}_"\
        f"{c['strand']}_{c['protospacer']}_{c['intended_target_name']}"
    else:
        return f"{c['type']}_{c['protospacer']}"
        
df.index = [get_index_str(f[1]) for f in df.iterrows()]


with open("~/data/TF_CRISPR_i_a.scRNAseq_screen.protospaces.fa","w") as ofile:
    for l in zip(df.index, df.protospacer):
        ofile.write(">" + l[0] + "\n")
        ofile.write(l[1] + "\n")

In [113]:
!git clone https://github.com/alejandrogzi/bed2gtf.git

Cloning into 'bed2gtf'...
remote: Enumerating objects: 313, done.
remote: Counting objects: 100% (142/142), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 313 (delta 75), reused 113 (delta 50), pack-reused 171
Receiving objects: 100% (313/313), 86.25 KiB | 1.69 MiB/s, done.
Resolving deltas: 100% (164/164), done.


Create GTF

In [15]:
%%bash
head ~/data/TF_CRISPR_i_a.scRNAseq_screen.protospaces.fa \
    | awk -vOFS="\t" 'NR%2==1{gsub(/>/,"", $1); print $1, "awkFa2GTF", "gene", 1, 21, ".", "+", ".", "gene_id " "\""$1"\""; print $1, "awkFa2GTF", "transcript", 1, 21, ".", "+", ".", "gene_id " "\""$1"\"; transcript_id \""$1"_trans\""; print $1, "awkFa2GTF", "exon", 1, 21, ".", "+", ".", "gene_id " "\""$1"\"; transcript_id \""$1"_trans\"; exon_id \""$1"_exon\"";}' \
> ~/data/TF_CRISPR_i_a.scRNAseq_screen.protospaces.gtf


In [16]:
!STAR --runThreadN 2 \
    --runMode genomeGenerate \
    --genomeDir ~/data/star_ref  \
    --genomeFastaFiles /home/jovyan/data/TF_CRISPR_i_a.scRNAseq_screen.protospaces.fa \
    --genomeSAindexNbases 3 \
    --sjdbGTFfile /home/jovyan/data/TF_CRISPR_i_a.scRNAseq_screen.protospaces.gtf


	/opt/conda/bin/STAR-avx2 --runThreadN 2 --runMode genomeGenerate --genomeDir /home/jovyan/data/star_ref --genomeFastaFiles /home/jovyan/data/TF_CRISPR_i_a.scRNAseq_screen.protospaces.fa --genomeSAindexNbases 3 --sjdbGTFfile /home/jovyan/data/TF_CRISPR_i_a.scRNAseq_screen.protospaces.gtf
	STAR version: 2.7.11b   compiled: 2024-01-29T15:15:38+0000 :/opt/conda/conda-bld/star_1706541070242/work/source
Feb 23 22:07:32 ..... started STAR run
Feb 23 22:07:32 ... starting to generate Genome files
Feb 23 22:07:32 ..... processing annotations GTF
Feb 23 22:07:32 ... starting to sort Suffix Array. This may take a long time...
Feb 23 22:07:32 ... sorting Suffix Array chunks and saving them to disk...
Feb 23 22:07:32 ... loading chunks from disk, packing SA...
Feb 23 22:07:32 ... finished generating suffix array
Feb 23 22:07:32 ... generating Suffix Array index
Feb 23 22:07:32 ... completed Suffix Array index
Feb 23 22:07:32 ... writing Genome to disk ...
Feb 23 22:07:32 ... writing Suffix Array t

In [17]:
!zcat  /mnt/shared/counting_guides/fastq/gRNA_CRISPRi_D3_S9_L001_R1_001.fastq.gz  |  head -n 400000 >/home/jovyan/data/test_R1_001.fastq
!zcat /mnt/shared/counting_guides/fastq/gRNA_CRISPRi_D3_S9_L001_R2_001.fastq.gz | head -n 400000 > /home/jovyan/data/test_R2_001.fastq


gzip: stdout: Broken pipe

gzip: stdout: Broken pipe


In [19]:
%%bash
mkdir -p /home/jovyan/star_solo
cd /home/jovyan/star_solo
STAR --soloType CB_UMI_Simple \
--readFilesIn  \
    /home/jovyan/data/test_R1_001.fastq  \
    /home/jovyan/data/test_R2_001.fastq    \
--genomeDir /home/jovyan/data/star_ref  \
--soloCBwhitelist /mnt/shared/counting_guides/metadata/737K-august-2016.txt   \
--soloCBstart 1 \
--soloCBlen 16 \
--soloUMIstart 17 \
--soloUMIlen 10 \
--soloBarcodeMate 1 \
--soloCBmatchWLtype 1MM_multi_Nbase_pseudocounts \
--soloUMIfiltering MultiGeneUMI_CR \
--soloUMIdedup 1MM_CR \
--outFilterScoreMin 20 \
--outSAMtype BAM SortedByCoordinate \
--outSAMattributes CR UR CY UY CB UB \
--soloBarcodeReadLength 0 \
--clip5pNbases 39 0  \
--outFilterScoreMinOverLread 0  --outFilterMatchNminOverLread 0 \


	/opt/conda/bin/STAR-avx2 --soloType CB_UMI_Simple --readFilesIn /home/jovyan/data/test_R1_001.fastq /home/jovyan/data/test_R2_001.fastq --genomeDir /home/jovyan/data/star_ref --soloCBwhitelist /mnt/shared/counting_guides/metadata/737K-august-2016.txt --soloCBstart 1 --soloCBlen 16 --soloUMIstart 17 --soloUMIlen 10 --soloBarcodeMate 1 --soloCBmatchWLtype 1MM_multi_Nbase_pseudocounts --soloUMIfiltering MultiGeneUMI_CR --soloUMIdedup 1MM_CR --outFilterScoreMin 20 --outSAMtype BAM SortedByCoordinate --outSAMattributes CR UR CY UY CB UB --soloBarcodeReadLength 0 --clip5pNbases 39 0 --outFilterScoreMinOverLread 0 --outFilterMatchNminOverLread 0
	STAR version: 2.7.11b   compiled: 2024-01-29T15:15:38+0000 :/opt/conda/conda-bld/star_1706541070242/work/source
Feb 23 22:12:54 ..... started STAR run
Feb 23 22:12:54 ..... loading genome
Feb 23 22:12:54 ..... started mapping
Feb 23 22:13:04 ..... finished mapping
Feb 23 22:13:04 ..... started Solo counting
Feb 23 22:13:05 ..... finished Solo counti

In [109]:
!head /mnt/shared/counting_guides/metadata/TF_CRISPR_i_a.scRNAseq_screen.igvf_metadata.txt

protospacer	targeting	type	guide_chr	guide_start	guide_end	strand	pam	genomic_element	intended_target_name	intended_target_start	intended_target_end	putative_target_genes	reporter	imperfect	pgRNA
CGGGGACACTGGGCCGACGCG	True	targeting	chr1	212699914.0	212699940.0	-	TGGAGT	promoter	BATF3	212699840.0	212699841.0				
CCGGTGGCCTTATTATAAAGG	True	targeting	chr3	27722356.0	27722355.0	+	CAGGAT	promoter	EOMES	27722711.0	27722712.0				
ACATATCTGGGAAAAGACCTA	True	targeting	chr14	75522341.0	75522367.0	-	CAGAAT	promoter	BATF	75522454.0	75522455.0				
CAGTTGAGGCTTGAAGGGCCA	True	targeting	chr3	4979808.0	4979834.0	-	AGGAGT	promoter	BHLHE40	4979436.0	4979437.0				
TCCGCCCATGTGACTTCCAGC	True	targeting	chr14	75522422.0	75522421.0	+	GTGAGT	promoter	BATF	75522454.0	75522455.0				
TTCCCCCACCACCAGCGGCGA	True	targeting	chr16	75148548.0	75148547.0	+	GTGGGT	promoter	ZFP1	75148493.0	75148494.0				
TCCAGCGTGTGAGCCTGGGAG	True	targeting	chr3	27722586.0	27722612.0	-	GAGGGT	promoter	EOMES	27722711.0	27722712.0				
TGTT

In [111]:
!gzip -dc /mnt/shared/counting_guides/fastq/gRNA_CRISPRi_D3_S9_L001_R2_001.fastq.gz | grep CGGGGACACTGGGCCGACGCG | head